Setting up an AlexNet in Keras. This is a relatively early network design, but goes quite deep compared to a multilayer perceptron.

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load up the CIFAR images, normalize the images on all color channels 0-1, and one hot encode the labels.

In [2]:
num_outputs = 10 # 10 output digits
batch_size = 128 # mini batch
epochs = 10 # total training loops
learning_rate = 0.01 # amount we update parameters

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = np.expand_dims(train_images / np.max(train_images), -1)
test_images = np.expand_dims(test_images / np.max(test_images), -1)
train_labels = keras.utils.to_categorical(train_labels, num_outputs)
test_labels = keras.utils.to_categorical(test_labels, num_outputs)

And now for our actual model. AlexNet was one of the original deep models that led to the resurgence of neural network techniques in machine learning. It had several innovations -- it was quite deep -- and it combined convolution with attenation -- merging together pixes with convolution, but adding more neural network depth.

We'll create these networks without special techniques like dropout or batch normalization to get a simplified view.

In [4]:
convolution_kernels = [(11, 11), (5, 5), (3, 3), (3, 3), (3, 3)]
convolution_filters = [96, 256, 512, 1024, 1024]
convolutional_pooling = (2, 2)
dense_units = [4096, 4096]
image_shape = train_images.shape[1:]

Using a few loops to stack up the deep layers. This lets you get a sense that making a deep network is just about layering.

We'll put in one placeholder layer to contain the image shape extracted frome the training data.

Note the use of `same` padding. This actually will pad the images. We need to do this here so that the input image is in fact big enough to 'divide' this many times. You'll see we in the final convolution we end up with a very small x and y dimension.


In [6]:
alexnet = Sequential()
alexnet.add(Reshape(image_shape[:-1], input_shape=image_shape))
for kernel, filters in zip(convolution_kernels, convolution_filters):
    alexnet.add(Conv2D(filters, kernel, activation='relu', padding='same'))
    alexnet.add(MaxPooling2D(convolutional_pooling, padding='same'))
for units in dense_units:
    alexnet.add(Dense(units, activation='relu'))
alexnet.add(Flatten())
alexnet.add(Dense(num_outputs, activation='softmax'))
alexnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 512)         1180160   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 512)         0         
__________

And as always, learning is done with an optimizer and a loss function, learning a classifier with categorical cross entropy.

In [7]:
optimizer = keras.optimizers.SGD(lr=learning_rate)
loss = keras.losses.categorical_crossentropy

Now, keep in mind this is starting to be a pretty big model. If you train this on a CPU, it is *possible*, but it is going to take a very long time. I'm running on a GPU

In [8]:
alexnet.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'])

history = alexnet.fit(train_images, train_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 22s 442us/step - loss: 2.2970 - acc: 0.1295 - val_loss: 2.2837 - val_acc: 0.1030
Epoch 2/10
50000/50000 [==============================] - 14s 285us/step - loss: 2.1837 - acc: 0.1849 - val_loss: 2.0200 - val_acc: 0.2650
Epoch 3/10
50000/50000 [==============================] - 14s 285us/step - loss: 2.0013 - acc: 0.2668 - val_loss: 1.9353 - val_acc: 0.3042
Epoch 4/10
50000/50000 [==============================] - 14s 288us/step - loss: 1.9060 - acc: 0.3111 - val_loss: 1.8214 - val_acc: 0.3516
Epoch 5/10
50000/50000 [==============================] - 15s 293us/step - loss: 1.8126 - acc: 0.3464 - val_loss: 1.9184 - val_acc: 0.3015
Epoch 6/10
50000/50000 [==============================] - 14s 286us/step - loss: 1.7209 - acc: 0.3807 - val_loss: 1.6441 - val_acc: 0.4062
Epoch 7/10
50000/50000 [==============================] - 14s 285us/step - loss: 1.6388 - acc: 0.4090 - val_loss: 1.

So this isn't mind boggling accurate, but it is a very complex problem to recognize open images. We can see this model kept learning on each epoch, and didn't appear to overfit. You should as an experiment, increase the number of epochs.